# PURPOSE OF THIS NOTEBOOK
This is the base code to write the functions to extract the raw text from resumes and append to our dataframe. 

TODO: Take the functions into a .py folder and use it as a script 

In [1]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
from tokenizers import Tokenizer
from transformers import BertTokenizer, BertModel
from pathlib import Path
from torch import cuda
import torch

from sklearn import metrics
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

C:\Users\zaffa\AppData\Local\Temp\ipykernel_23308\478442376.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Setting up for GPU 

device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
torch.cuda.empty_cache()

## Model Goal

The goal is to train a custom BERT model to attempt to label each incoming job title to their proper ONET code. ONET (Occupational Information Network) is a free database that serves as a standardized taxonomy for jobs. Each job has a respective standardized name and code associated with it. This will make extracting skills quite easy if we can corrrectly translate job titles to the proper ONET code. Using BERT, we can tokenize the job titles and match them with the database of common job titles for each ONET code. For this first version I will be leaving out the actual ONET job names from the training data to compare later with an updated dataset. More information about ONET can be found here: https://www.onetonline.org/

In [4]:
# Import the train/test Data. 
test_df = pd.read_csv("../Data/TestingData.csv")
train_df = pd.read_csv("../Data/Training_Data.csv")
label_df = pd.read_csv("../Data/label_df.csv")

In [5]:
# Check that the test data incoming is correct
test_df.head()

,Reported_Jobs,Label
0,General and Operations Managers,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Marketing Managers,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Public Relations Managers,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Financial Managers,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Treasurers and Controllers,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
# Check that the training data incoming is correct 
train_df.head()

,Reported_Jobs,Label
0,"Farmers, Ranchers, and Other Agricultural Mana...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Supply Chain Planning Manager,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Sales Vice President (Sales VP),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Special Programs Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Offshore Wind Operations Manager,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
train_df['Label'] = train_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])

In [8]:
test_df['Label'] = test_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])

## The goal of the tokenizer

From looking at the reference code, I've learned that we need to follow these steps: 
1. Start with a train test split. **70% for the training data**, I will do the split based on the **70%** of each reported job title/ONET pairing.
    - Given the refernece notebook uses a dictionary as the input data and we are working with a dataframe instead, some major changes will be needed to be made in order for this to work. I don't think this would be difficult at all. Just need to translate the dictionary work to the dataframe. **I also need to confirm if the model input requires a list, dict, or dataframe object.**
2. Run the tokenizer on the training set 
3. Set up the model training and evaluation metrics.


In [9]:
# Load in the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [10]:
MAX_LEN = 175
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 30
LEARNING_RATE = 2e-6

In [11]:
# Start by tokenizing the data.
# Will be using the class statment and slowly converting it for our needs 
class CustomDataset(Dataset):
    
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.reported_jobs = dataframe.Reported_Jobs
        self.targets = self.data.Label
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reported_jobs)
    
    def __getitem__(self, index):
        reported_job = str(self.reported_jobs[index])
        reported_job = " ".join(reported_job.split())

        inputs = self.tokenizer.encode_plus(
            reported_job,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [12]:
# Running the tokenizer and shaping the dataframes for the model
train_set = CustomDataset(train_df, tokenizer, MAX_LEN)
test_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [13]:
# Setting the train and test parameters 
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0}

test_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': True,
               'num_workers': 0}

training_loader = DataLoader(train_set, **train_params)
testing_loader = DataLoader(test_set, **test_params)

In [14]:
# Creatring the custom model

class BERTClass(torch.nn.Module):
    def __init__(self):
        # Defining the layers
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 22)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output_3 = self.l3(output_2)
        return output_3

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [15]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

# Fine tuning

In [16]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss= loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch : {epoch}, Loss: {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return outputs

In [17]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _,data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            
            outputs = model(ids, mask, token_type_ids)

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [18]:
for epoch in range(EPOCHS):
    output = train(epoch)
    # evaluate
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\zaffa\.virtualenvs\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch : 0, Loss: 0.6686908602714539
Epoch : 0, Loss: 0.5447896718978882
Epoch : 0, Loss: 0.42241349816322327
Epoch : 0, Loss: 0.3650408685207367
Epoch : 0, Loss: 0.31953945755958557
Epoch : 0, Loss: 0.2928646206855774
Epoch : 0, Loss: 0.25403958559036255
Epoch : 0, Loss: 0.234996497631073
Epoch : 0, Loss: 0.21643680334091187
Epoch : 0, Loss: 0.22354571521282196
Epoch : 0, Loss: 0.22798292338848114
Epoch : 0, Loss: 0.21032732725143433
Epoch : 0, Loss: 0.21259330213069916
Epoch : 0, Loss: 0.19307245314121246
Epoch : 0, Loss: 0.17451123893260956
Epoch : 0, Loss: 0.19305419921875
Epoch : 0, Loss: 0.2032051980495453
Epoch : 0, Loss: 0.1881873905658722
Epoch : 0, Loss: 0.17739985883235931
Epoch : 0, Loss: 0.16108138859272003
Epoch : 0, Loss: 0.18719148635864258
Epoch : 0, Loss: 0.14387093484401703
Epoch : 0, Loss: 0.15119338035583496
Epoch : 0, Loss: 0.1782234013080597
Epoch : 0, Loss: 0.1684022694826126
Epoch : 0, Loss: 0.15253818035125732
Epoch : 0, Loss: 0.17267084121704102
Epoch : 0, Los

0it [00:00, ?it/s]

Accuracy Score = 0.2526326986490799
F1 Score (Micro) = 0.3770589402659258
F1 Score (Macro) = 0.03822582106663401


...

Epoch : 29, Loss: 0.0052626412361860275


c:\Users\zaffa\.virtualenvs\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch : 29, Loss: 0.0031354522798210382
Epoch : 29, Loss: 0.053498100489377975
Epoch : 29, Loss: 0.025534169748425484
Epoch : 29, Loss: 0.0015832858625799417
Epoch : 29, Loss: 0.013857810758054256
Epoch : 29, Loss: 0.005263825412839651
Epoch : 29, Loss: 0.05126305669546127
Epoch : 29, Loss: 0.006198142189532518
Epoch : 29, Loss: 0.018132701516151428
Epoch : 29, Loss: 0.003210870549082756
Epoch : 29, Loss: 0.014960629865527153
Epoch : 29, Loss: 0.048801880329847336
Epoch : 29, Loss: 0.004973267205059528
Epoch : 29, Loss: 0.03718138486146927
Epoch : 29, Loss: 0.0025313813239336014
Epoch : 29, Loss: 0.03009159304201603
Epoch : 29, Loss: 0.007633702363818884
Epoch : 29, Loss: 0.031042929738759995
Epoch : 29, Loss: 0.026680169627070427
Epoch : 29, Loss: 0.0026581494603306055
Epoch : 29, Loss: 0.003589618019759655
Epoch : 29, Loss: 0.010899747721850872
Epoch : 29, Loss: 0.00653076171875
Epoch : 29, Loss: 0.08562634885311127
Epoch : 29, Loss: 0.008838595822453499
Epoch : 29, Loss: 0.036549162

0it [00:00, ?it/s]

Accuracy Score = 0.776832251888097
F1 Score (Micro) = 0.7928713724223334
F1 Score (Macro) = 0.7706064931098272


## Model Evaluation and testing 

In [19]:
def predict(input, tokenizer, model, device):
    ''' GOAL OF THIS FUNCTION: 
    This function takes in any given string and converts it into a tokenized version that can be run through the model. '''

    input = " ".join(input.split())
    inputs = tokenizer.encode_plus(
        input,
        None,
        add_special_tokens=True,
        max_length=175,
        pad_to_max_length=True,
        return_token_type_ids=True
    )
    
    ids = torch.tensor(inputs['input_ids'], dtype=torch.long).to(device, dtype = torch.long)
    mask = torch.tensor(inputs['attention_mask'], dtype=torch.long).to(device, dtype= torch.long)
    token_type_ids = torch.tensor(inputs['token_type_ids'], dtype=torch.long).to(device, dtype= torch.long)

    output = model(ids.unsqueeze(0), mask.unsqueeze(0), token_type_ids.unsqueeze(0))

    return output

## Test Predictions with the model 

In [29]:
# To predict, the model is looking for the ids, mask and token_type_ids. Which means whatever we input must be run through the tokenizer first. 
input='Data Scientist'
output = predict(input, tokenizer, model, device)

c:\Users\zaffa\.virtualenvs\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [30]:
output_df = pd.DataFrame(output.cpu().detach().numpy().tolist(), columns=label_df.columns)

In [31]:
output_df

,11,13,15,17,19,21,23,25,27,29,...,35,37,39,41,43,45,47,49,51,53
0,-7.454539,-6.445076,-0.202713,-5.343482,0.482879,-6.678531,-8.999181,-7.400536,-7.201194,-6.639618,...,-7.614067,-7.407125,-8.344387,-8.23388,-6.436518,-7.297492,-7.321807,-6.37388,-6.417892,-7.336663


In [32]:
# Checking the prediction, which is the value with the highest value. 
output_df.idxmax(axis='columns')

0    19
dtype: object

In [24]:
# Export model: 
torch.save(model, f'../Data/Models/ONET_Family_Model')

## MODEL TRAINING PART TWO
Now we train a model that will predict the specific job that this title belongs too. This will have to be 22 indidvidual models that will run based on the group that is chosen. I'm a little worried that the performance will not be as strong in comparison of the main model given that the data for each major group is much smaller in comparison to the main dataset 

In [25]:
model = torch.load("../Data/Models/ONET_Family_Model")
model.eval

<bound method Module.eval of BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [26]:
input='Data Engineer'
output = predict(input, tokenizer, model, device)

c:\Users\zaffa\.virtualenvs\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [27]:
output_df = pd.DataFrame(output.cpu().detach().numpy().tolist(), columns=label_df.columns)

In [28]:
output_df.idxmax(axis='columns')

0    15
dtype: object